In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from pathlib import Path
from copy import deepcopy
from tqdm import tqdm

In [2]:
if torch.cuda.is_available():
    print("GPU is available!")
    device = torch.device("cuda:0")
else:
    print("WARNING: Could not find GPU! Using CPU only")
    device = torch.device("cpu")
print(f"Using device: {device}")

GPU is available!
Using device: cuda:0


In [3]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2
batch_size = 2

In [4]:
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.brnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        out, _ = self.brnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [5]:
# Load Data
train_dataset = datasets.MNIST(
    root="mnst_files/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = datasets.MNIST(
    root="mnst_files/", train=False, transform=transforms.ToTensor(), download=True
)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
<urlopen error [Errno -3] Temporary failure in name resolution>



100%|██████████| 9912422/9912422 [00:00<00:00, 16955781.24it/s]


Extracting mnst_files/MNIST/raw/train-images-idx3-ubyte.gz to mnst_files/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno -5] No address associated with hostname>



100%|██████████| 28881/28881 [00:00<00:00, 487534.29it/s]


Extracting mnst_files/MNIST/raw/train-labels-idx1-ubyte.gz to mnst_files/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno -5] No address associated with hostname>



100%|██████████| 1648877/1648877 [00:00<00:00, 4548405.43it/s]


Extracting mnst_files/MNIST/raw/t10k-images-idx3-ubyte.gz to mnst_files/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno -5] No address associated with hostname>



100%|██████████| 4542/4542 [00:00<00:00, 8519914.48it/s]

Extracting mnst_files/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnst_files/MNIST/raw



In [6]:
# Initialize network
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

 15%|█▌        | 4619/30000 [00:27<02:31, 167.62it/s]


KeyboardInterrupt: 

In [ ]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy  \
              {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)